In [ ]:
#featurelar çıkarılacak
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import re
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

# Example DataFrame
data = pd.read_excel('/content/project_dataset_analysed2.xlsx')

df = pd.DataFrame(data)

In [ ]:
df

,Fiyat,İlan No,Marka,Seri,Model,Yıl,Km,Vites Tipi,Yakıt Tipi,Kasa Tipi,...,IYI OZELLIK,KOTU OZELLIK,BILGI YOK,IYI_OZELLIK_NUMERIC,KOTU_OZELLIK_NUMERIC,TRAMER NUMERİK,BAKIM_DURUMU_BOOL,BOYA SAYISI NUMERİC,DEGISEN SAYISI NUMERİC,YEDEK ANAHTAR BOOL
0,500000,24113903,Citroen,C-Elysee,1.6 HDi Attraction,2015,300000,Düz,Dizel,Sedan,...,[],[],[],0,0,0,0,0,0,0
1,541000,24105755,Renault,Symbol,1.5 dCi Joy,2016,123500,Düz,Dizel,Sedan,...,[],[],[],0,0,0,0,0,0,0
2,320000,24103903,Citroen,C3,1.4 HDi Elit,2005,360000,Düz,Dizel,Hatchback/5,...,[],[],[],0,0,0,0,1,0,0
3,390000,24118518,Citroen,C4,1.6 HDi SX,2005,187000,Düz,Dizel,Hatchback/5,...,[],[],[],0,0,0,0,0,0,0
4,550000,24116287,Citroen,C4,1.6 HDi Confort,2012,185000,Düz,Dizel,Hatchback/5,...,[],[],[],0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219,645000,24150138,Volkswagen,Polo,1.4 TDi Comfortline,2017,153000,Yarı Otomatik,Dizel,Hatchback/5,...,['degisen yok'],[],[],1,0,0,0,2,0,0
10220,630000,24106909,Opel,Corsa,1.4 Twinport Color Edition,2011,145000,Otomatik,Benzin,Hatchback/5,...,[],[],[],0,0,0,1,0,0,0
10221,165000,23633877,Renault,R 9,1.4 GT,1988,333333,Düz,LPG & Benzin,Sedan,...,[],[],[],0,0,0,0,1,0,0
10222,320000,24110038,Renault,Symbol,1.5 dCi Joy,2014,130000,Düz,Dizel,Sedan,...,[],[],[],0,0,0,0,0,2,0


Change the paint and its changing features that are not specified on the website with the help of the features from NER model

In [ ]:
def convert_value(value,a_value):
    if value == 'Yok':
        return 0
    elif value == 'Belirtilmemiş':
        return a_value
    elif value == 'Tamamı boyalı' or value == 'Tamamı değişen' :
        return 25
    else:
        match = re.search(r'\d+', value)
        if match:
            return int(match.group(0))
        else:
            return value

In [ ]:
df['Değişen'] = df.apply(lambda row: convert_value(row['Değişen'], row['DEGISEN SAYISI NUMERİC']), axis=1)
df['Boya'] = df.apply(lambda row: convert_value(row['Boya'], row['BOYA SAYISI NUMERİC']), axis=1)

In [ ]:
X = df.drop(columns=['İlan No','Boya-değişen', 'Kimden', 'Açıklama', 'Processed Açıklama Metni','DEGISEN', 'TRAMER', 'YEDEK ANAHTAR', 'BOYA',
       'BAKIM DURUMU', 'DEGISEN YOK', 'IYI OZELLIK', 'KOTU OZELLIK',
       'BILGI YOK','DEGISEN SAYISI NUMERİC','BOYA SAYISI NUMERİC',
       ])  # Features

In [ ]:
X

,Fiyat,Marka,Seri,Model,Yıl,Km,Vites Tipi,Yakıt Tipi,Kasa Tipi,Renk,...,Yakıt Deposu,Değişen,Boya,sentiment,sent-scores,IYI_OZELLIK_NUMERIC,KOTU_OZELLIK_NUMERIC,TRAMER NUMERİK,BAKIM_DURUMU_BOOL,YEDEK ANAHTAR BOOL
0,500000,Citroen,C-Elysee,1.6 HDi Attraction,2015,300000,Düz,Dizel,Sedan,Beyaz,...,48,0,4,negative,0.999712,0,0,0,0,0
1,541000,Renault,Symbol,1.5 dCi Joy,2016,123500,Düz,Dizel,Sedan,Beyaz,...,50,0,1,negative,0.999661,0,0,0,0,0
2,320000,Citroen,C3,1.4 HDi Elit,2005,360000,Düz,Dizel,Hatchback/5,Yeşil (metalik),...,45,0,12,negative,0.999650,0,0,0,0,0
3,390000,Citroen,C4,1.6 HDi SX,2005,187000,Düz,Dizel,Hatchback/5,Gri (Gümüş),...,60,0,5,negative,0.999614,0,0,0,0,0
4,550000,Citroen,C4,1.6 HDi Confort,2012,185000,Düz,Dizel,Hatchback/5,Beyaz,...,60,0,0,negative,0.999511,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219,645000,Volkswagen,Polo,1.4 TDi Comfortline,2017,153000,Yarı Otomatik,Dizel,Hatchback/5,Beyaz,...,45,0,2,positive,0.500601,1,0,0,0,0
10220,630000,Opel,Corsa,1.4 Twinport Color Edition,2011,145000,Otomatik,Benzin,Hatchback/5,Kırmızı,...,45,0,0,positive,0.500420,0,0,0,1,0
10221,165000,Renault,R 9,1.4 GT,1988,333333,Düz,LPG & Benzin,Sedan,Füme,...,60,0,1,negative,0.500386,0,0,0,0,0
10222,320000,Renault,Symbol,1.5 dCi Joy,2014,130000,Düz,Dizel,Sedan,Beyaz,...,50,0,0,negative,0.500190,0,0,0,0,0


In [ ]:

# Encode categorical features
label_encoders = {}
for column in ['Marka','Seri','Model', 'Vites Tipi', 'Yakıt Tipi', 'Kasa Tipi','Renk','sentiment','Çekiş']:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le

# Normalize numerical features
scaler = StandardScaler()
X[['Fiyat', 'Yıl', 'Km','Motor Hacmi','Motor Gücü','Ort. Yakıt Tüketimi','Yakıt Deposu', 'Değişen','Boya','IYI_OZELLIK_NUMERIC','TRAMER NUMERİK','sent-scores', 'KOTU_OZELLIK_NUMERIC',
        'BAKIM_DURUMU_BOOL', 'YEDEK ANAHTAR BOOL']] = scaler.fit_transform(X[['Fiyat', 'Yıl', 'Km','Motor Hacmi','Motor Gücü','Ort. Yakıt Tüketimi','Yakıt Deposu', 'Değişen','Boya','IYI_OZELLIK_NUMERIC','TRAMER NUMERİK','sent-scores', 'KOTU_OZELLIK_NUMERIC',
        'BAKIM_DURUMU_BOOL', 'YEDEK ANAHTAR BOOL']] )


In [ ]:
X

,Fiyat,Marka,Seri,Model,Yıl,Km,Vites Tipi,Yakıt Tipi,Kasa Tipi,Renk,...,Yakıt Deposu,Değişen,Boya,sentiment,sent-scores,IYI_OZELLIK_NUMERIC,KOTU_OZELLIK_NUMERIC,TRAMER NUMERİK,BAKIM_DURUMU_BOOL,YEDEK ANAHTAR BOOL
0,-0.037414,6,89,887,0.647408,0.106610,0,1,8,2,...,-0.545957,-0.536119,0.500488,0,1.136378,-0.470165,-0.215749,-0.023756,-0.419020,-0.352502
1,-0.031669,32,266,690,0.771014,-0.070238,0,1,8,2,...,-0.286595,-0.536119,-0.295807,0,1.136025,-0.470165,-0.215749,-0.023756,-0.419020,-0.352502
2,-0.062633,6,93,403,-0.588652,0.166728,0,1,3,22,...,-0.935000,-0.536119,2.623941,0,1.135948,-0.470165,-0.215749,-0.023756,-0.419020,-0.352502
3,-0.052825,6,95,907,-0.588652,-0.006613,0,1,3,7,...,1.010216,-0.536119,0.765920,0,1.135702,-0.470165,-0.215749,-0.023756,-0.419020,-0.352502
4,-0.030408,6,95,890,0.276590,-0.008617,0,1,3,2,...,1.010216,-0.536119,-0.561238,0,1.134996,-0.470165,-0.215749,-0.023756,-0.419020,-0.352502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10219,-0.017098,42,214,501,0.894620,-0.040680,2,1,3,2,...,-0.935000,-0.536119,-0.030375,1,-2.302902,0.109896,-0.215749,-0.023756,-0.419020,-0.352502
10220,-0.019200,28,119,541,0.152984,-0.048695,1,0,3,11,...,-0.935000,-0.536119,-0.561238,1,-2.304145,-0.470165,-0.215749,-0.023756,2.386521,-0.352502
10221,-0.084349,32,224,395,-2.689953,0.140008,0,4,8,5,...,1.010216,-0.536119,-0.295807,0,-2.304384,-0.470165,-0.215749,-0.023756,-0.419020,-0.352502
10222,-0.062633,32,266,690,0.523802,-0.063725,0,1,8,2,...,-0.286595,-0.536119,-0.561238,0,-2.305729,-0.470165,-0.215749,-0.023756,-0.419020,-0.352502


Build a function that clusters that data and evaluates the model

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score
import numpy as np

def clustering(model, data):
    # Modeli fit et ve tahmin etiketlerini al
    label = model.fit_predict(data)
    cluster_labels = model.labels_

    # Küme merkezlerini al
    cluster_centers = model.cluster_centers_

    # Küme merkezlerini yazdır
    print("Cluster labels:", cluster_labels)
    print("Cluster centers:", cluster_centers)

    # Silhouette ve Davies-Bouldin skorlarını hesapla
    silhouette = silhouette_score(data, label)
    davies_bouldin = davies_bouldin_score(data, label)

    # Silhouette ve Davies-Bouldin skorlarını yazdır
    print("Silhouette Score:", silhouette)
    print("Davies-Bouldin Score:", davies_bouldin)

    # Her kümeye ait eleman sayılarını yazdır
    unique_labels, counts = np.unique(cluster_labels, return_counts=True)
    print("Her kümeye ait eleman sayısı:")
    for label, count in zip(unique_labels, counts):
        print(f"Küme {label}: {count} eleman")

    return label, model

Use Function

In [ ]:
model = KMeans(n_clusters=3)
data = X
all_data_modeled,all_data_model = clustering(model,data)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster labels: [2 2 0 ... 0 2 2]
Cluster centers: [[-2.26217986e-02  2.56407895e+01  1.54083158e+02  3.05500526e+02
   1.54285265e-01 -2.81150616e-03  4.08157895e-01  1.53236842e+00
   5.06736842e+00  6.59684211e+00 -7.15227811e-01 -4.21376531e-01
   2.99342105e+00 -1.15647784e-01 -5.49403399e-01  2.25601447e-02
  -5.02126126e-02  5.26315789e-01  8.35074044e-03 -1.16785542e-01
   2.23655614e-02  6.33297282e-03 -1.66464167e-02  2.35071472e-02]
 [ 5.78403255e-02  1.90070838e+01  1.16393152e+02  1.48845514e+03
  -4.43155624e-01  1.21413706e-02  7.27863046e-01  1.88252656e+00
   6.55489965e+00  8.99586777e+00  1.03499639e+00  8.85619164e-01
   2.30460449e+00  4.44705983e-01  6.79429567e-01 -1.00977605e-01
   2.71298609e-02  6.33412043e-01  3.42499099e-02  3.10554659e-01
  -6.42759865e-02 -2.12644523e-02 -1.30847599e-01 -7.76218451e-02]
 [-2.54094647e-03  2.68775899e+01  1.51444186e+02  8.71957928e+02
   3.47614417e-02 -2.08958953e-03  6.11627907e-01  2.01078224e+00
   6.27463002e+00  6.75

Use Function

In [ ]:
model = KMeans(n_clusters=3)
data = X.drop(columns=['sent-scores', 'KOTU_OZELLIK_NUMERIC',
        'BAKIM_DURUMU_BOOL', 'YEDEK ANAHTAR BOOL'])
all_data_modeled,all_data_model = clustering(model,data)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster labels: [0 0 2 ... 2 0 0]
Cluster centers: [[-2.54094647e-03  2.68775899e+01  1.51444186e+02  8.71957928e+02
   3.47614417e-02 -2.08958953e-03  6.11627907e-01  2.01078224e+00
   6.27463002e+00  6.75835095e+00  2.03928497e-01  2.13513644e-02
   2.97167019e+00 -6.63573686e-02  1.98050577e-01  1.80396432e-02
   3.06236668e-02  5.22621564e-01 -1.73984211e-02  2.52784048e-03]
 [ 5.78403255e-02  1.90070838e+01  1.16393152e+02  1.48845514e+03
  -4.43155624e-01  1.21413706e-02  7.27863046e-01  1.88252656e+00
   6.55489965e+00  8.99586777e+00  1.03499639e+00  8.85619164e-01
   2.30460449e+00  4.44705983e-01  6.79429567e-01 -1.00977605e-01
   2.71298609e-02  6.33412043e-01  3.10554659e-01 -2.12644523e-02]
 [-2.26217986e-02  2.56407895e+01  1.54083158e+02  3.05500526e+02
   1.54285265e-01 -2.81150616e-03  4.08157895e-01  1.53236842e+00
   5.06736842e+00  6.59684211e+00 -7.15227811e-01 -4.21376531e-01
   2.99342105e+00 -1.15647784e-01 -5.49403399e-01  2.25601447e-02
  -5.02126126e-02  5.26

Use Function

In [ ]:
model = KMeans(n_clusters=3)
data = X.drop(columns=['sent-scores', 'KOTU_OZELLIK_NUMERIC',
        'BAKIM_DURUMU_BOOL', 'YEDEK ANAHTAR BOOL'])
all_data_modeled,all_data_model = clustering(model,data)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster labels: [2 2 0 ... 0 2 2]
Cluster centers: [[-2.26217986e-02  2.56407895e+01  1.54083158e+02  3.05500526e+02
   1.54285265e-01 -2.81150616e-03  4.08157895e-01  1.53236842e+00
   5.06736842e+00  6.59684211e+00 -7.15227811e-01 -4.21376531e-01
   2.99342105e+00 -1.15647784e-01 -5.49403399e-01  2.25601447e-02
  -5.02126126e-02  5.26315789e-01 -1.16785542e-01  6.33297282e-03]
 [ 5.78403255e-02  1.90070838e+01  1.16393152e+02  1.48845514e+03
  -4.43155624e-01  1.21413706e-02  7.27863046e-01  1.88252656e+00
   6.55489965e+00  8.99586777e+00  1.03499639e+00  8.85619164e-01
   2.30460449e+00  4.44705983e-01  6.79429567e-01 -1.00977605e-01
   2.71298609e-02  6.33412043e-01  3.10554659e-01 -2.12644523e-02]
 [-2.54094647e-03  2.68775899e+01  1.51444186e+02  8.71957928e+02
   3.47614417e-02 -2.08958953e-03  6.11627907e-01  2.01078224e+00
   6.27463002e+00  6.75835095e+00  2.03928497e-01  2.13513644e-02
   2.97167019e+00 -6.63573686e-02  1.98050577e-01  1.80396432e-02
   3.06236668e-02  5.22

Use Function

In [ ]:
model = KMeans(n_clusters=3)
data = X.drop(columns=['Marka','Seri','Model','sent-scores', 'KOTU_OZELLIK_NUMERIC',
        'BAKIM_DURUMU_BOOL', 'YEDEK ANAHTAR BOOL'])
weights = [0.6,0.4,0.3,0.1,0.3,0.7,0.2,0.1,0.1,0.2,0.2, 0.3,
       0.1,0.2,-0.4,-0.5,0.1,0.4,-0.3]
all_data_modeled,all_data_model = clustering(model,data)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster labels: [0 0 1 ... 0 0 0]
Cluster centers: [[-1.53542743e-02  7.83671707e-02 -8.62387443e-03  5.27622456e-01
   1.88973384e+00  8.04473272e+00  3.75352270e+00  5.87475221e-02
   5.09694364e-02  2.85528964e+00 -4.23436155e-02  1.28610962e-01
  -3.24266924e-02 -5.36471097e-02  5.42384254e-01 -6.09010765e-03
   1.07150243e-03]
 [-1.56824542e-03 -2.81884899e-01  4.42404719e-03  5.60741249e-01
   2.05113246e+00  5.42553191e+00  1.52590940e+01  1.56846436e-01
   1.20946898e-01  2.80713795e+00  2.76752485e-01  1.17579455e-01
   2.47419176e-02  1.27936540e-01  5.34660261e-01  3.18645874e-02
  -2.01166924e-02]
 [ 2.57903585e-02  1.65915102e-01  9.04039065e-03  5.93166608e-01
   1.44311377e+00  2.90982740e+00  3.88411412e+00 -2.53508519e-01
  -2.04411275e-01  2.95491370e+00 -2.17378808e-01 -3.23228652e-01
   2.56716428e-02 -4.68301695e-02  5.50193730e-01 -2.31153703e-02
   1.89606743e-02]]
Silhouette Score: 0.30993530631190463
Davies-Bouldin Score: 1.2549356183291387
Her kümeye ait elema

Use Function

In [ ]:
model = KMeans(n_clusters=3)
data = X.drop(columns=['Marka','Seri','Model'])
all_data_modeled,all_data_model = clustering(model,data)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster labels: [0 0 1 ... 0 0 0]
Cluster centers: [[-1.53542743e-02  7.83671707e-02 -8.62387443e-03  5.27622456e-01
   1.88973384e+00  8.04473272e+00  3.75352270e+00  5.87475221e-02
   5.09694364e-02  2.85528964e+00 -4.23436155e-02  1.28610962e-01
  -3.24266924e-02 -5.36471097e-02  5.42384254e-01  5.56908249e-03
  -6.09010765e-03 -1.63907909e-02  1.07150243e-03 -1.55392140e-02
  -6.52992331e-03]
 [-1.56834906e-03 -2.82246347e-01  4.46498408e-03  5.60933745e-01
   2.05149331e+00  5.42636457e+00  1.52608994e+01  1.57034752e-01
   1.21141136e-01  2.80707175e+00  2.77203078e-01  1.17940793e-01
   2.49344548e-02  1.28173126e-01  5.34500515e-01 -5.74386890e-03
   3.08421585e-02  1.14206825e-02 -2.01156565e-02 -9.69808154e-03
  -5.79813527e-02]
 [ 2.57808315e-02  1.66128165e-01  8.99677604e-03  5.92957746e-01
   1.44295775e+00  2.90985915e+00  3.88626761e+00 -2.53557184e-01
  -2.04495943e-01  2.95492958e+00 -2.17666993e-01 -3.23444064e-01
   2.54738292e-02 -4.70112991e-02  5.50352113e-01 -2.

Use Function

In [ ]:
model = KMeans(n_clusters=3)
data = X.drop(columns=['sent-scores', 'KOTU_OZELLIK_NUMERIC',
        'BAKIM_DURUMU_BOOL', 'YEDEK ANAHTAR BOOL','Çekiş'])
all_data_modeled,all_data_model = clustering(model,data)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster labels: [2 2 0 ... 0 2 2]
Cluster centers: [[-2.26217986e-02  2.56407895e+01  1.54083158e+02  3.05500526e+02
   1.54285265e-01 -2.81150616e-03  4.08157895e-01  1.53236842e+00
   5.06736842e+00  6.59684211e+00 -7.15227811e-01 -4.21376531e-01
  -1.15647784e-01 -5.49403399e-01  2.25601447e-02 -5.02126126e-02
   5.26315789e-01 -1.16785542e-01  6.33297282e-03]
 [ 5.77833019e-02  1.90171091e+01  1.16444838e+02  1.48827552e+03
  -4.43241462e-01  1.22674487e-02  7.28613569e-01  1.88200590e+00
   6.55280236e+00  8.99469027e+00  1.03522101e+00  8.85079555e-01
   4.44557982e-01  6.79242181e-01 -1.01234325e-01  2.75657251e-02
   6.33628319e-01  3.10094058e-01 -2.12659220e-02]
 [-2.53327597e-03  2.68756608e+01  1.51433073e+02  8.71891943e+02
   3.48932694e-02 -2.13778856e-03  6.11334320e-01  2.01099598e+00
   6.27532248e+00  6.75829985e+00  2.03672249e-01  2.13620159e-02
  -6.64123912e-02  1.98015948e-01  1.81568262e-02  3.04681802e-02
   5.22520617e-01 -1.73026786e-02  2.53339841e-03]]
Sil

Use Function

In [ ]:

model = KMeans(n_clusters=3)
data = X.drop(columns=['Marka','Seri'])
all_data_modeled,all_data_model = clustering(model,data)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster labels: [0 0 1 ... 1 0 0]
Cluster centers: [[-2.52826921e-03  8.71824239e+02  3.50774396e-02 -2.26505485e-03
   6.11463621e-01  2.01120981e+00  6.27580372e+00  6.75549915e+00
   2.03158997e-01  2.11322629e-02  2.97229272e+00 -6.66051278e-02
   1.97377882e-01  1.82740588e-02  3.13231542e-02  5.22208122e-01
  -1.84183542e-02 -1.72068956e-02  6.09719704e-03  2.53895869e-03
   5.98444830e-02  9.06710032e-03]
 [-2.26217986e-02  3.05500526e+02  1.54285265e-01 -2.81150616e-03
   4.08157895e-01  1.53236842e+00  5.06736842e+00  6.59684211e+00
  -7.15227811e-01 -4.21376531e-01  2.99342105e+00 -1.15647784e-01
  -5.49403399e-01  2.25601447e-02 -5.02126126e-02  5.26315789e-01
   8.35074044e-03 -1.16785542e-01  2.23655614e-02  6.33297282e-03
  -1.66464167e-02  2.35071472e-02]
 [ 5.77337803e-02  1.48810083e+03 -4.43472961e-01  1.26137398e-02
   7.28183962e-01  1.88148585e+00  6.55129717e+00  9.00117925e+00
   1.03616152e+00  8.85210778e-01  2.30365566e+00  4.44794001e-01
   6.80737200e-01 -1.

In [ ]:
X.columns

Index(['Fiyat', 'Marka', 'Seri', 'Model', 'Yıl', 'Km', 'Vites Tipi',
       'Yakıt Tipi', 'Kasa Tipi', 'Renk', 'Motor Hacmi', 'Motor Gücü', 'Çekiş',
       'Ort. Yakıt Tüketimi', 'Yakıt Deposu', 'Değişen', 'Boya', 'sentiment',
       'sent-scores', 'IYI_OZELLIK_NUMERIC', 'KOTU_OZELLIK_NUMERIC',
       'TRAMER NUMERİK', 'BAKIM_DURUMU_BOOL', 'YEDEK ANAHTAR BOOL'],
      dtype='object')

In [ ]:
import pickle

# Save model
with open('clustering_model.pkl', 'wb') as f:
    pickle.dump(all_data_model, f)

In [ ]:
with open('clustering_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

Load model

In [ ]:
loaded_model

KMeans(n_clusters=5)

use the model wiith other data

In [ ]:
c =X.iloc[:, :-1]

In [ ]:
son_15_veri_df = c.tail(15)

In [ ]:
tahminler = loaded_model.predict(son_15_veri_df)

In [ ]:
tahminler

array([4, 1, 4, 4, 1, 4, 0, 4, 0, 1, 2, 2, 2, 4, 4], dtype=int32)

In [ ]:
X.tail(15)

,Fiyat,Marka,Seri,Model,Yıl,Km,Vites Tipi,Yakıt Tipi,Kasa Tipi,Renk,Motor Hacmi,Motor Gücü,Ort. Yakıt Tüketimi,Yakıt Deposu,Değişen,Boya,sentiment,IYI_OZELLIK_NUMERIC,TRAMER NUMERİK,Cluster
10209,-0.062647,42,214,725,-1.330287,0.031462,0,4,3,2,0.338490,-0.860717,0.989513,-0.935000,-0.536119,-0.561238,1,-0.470165,-0.023756,4
10210,-0.047221,41,117,1069,-1.206682,-0.013626,0,4,8,7,0.338490,0.109597,1.061847,-0.286595,0.658580,-0.030375,1,-0.470165,-0.023756,1
10211,-0.033211,32,266,708,0.771014,-0.037674,0,1,8,18,-0.156749,-0.444868,-1.180492,-0.286595,0.658580,0.235056,0,-0.470165,0.008386,4
10212,0.130714,42,75,652,0.894620,-0.081760,2,0,3,2,-0.022999,1.218527,0.338512,1.788302,-0.536119,-0.561238,1,0.109896,-0.023756,4
10213,-0.005890,42,149,1047,0.400196,0.021442,2,1,3,18,0.338490,-0.029019,-1.180492,-0.286595,-0.536119,-0.561238,0,0.689957,-0.023756,1
10214,-0.018499,32,142,708,0.771014,-0.023646,2,1,8,18,-0.156749,0.109597,-0.818825,1.010216,-0.536119,-0.561238,1,-0.470165,-0.023756,4
10215,-0.083649,32,224,292,-2.442741,0.086570,0,4,8,2,-0.413407,-0.638931,-0.818825,-0.675638,-0.536119,-0.561238,0,-0.470165,-0.023756,0
10216,-0.020601,6,89,902,1.018226,-0.111819,0,1,8,2,0.201124,-0.361699,-0.891158,-0.545957,-0.536119,-0.561238,0,-0.470165,-0.023756,4
10217,-0.073141,14,141,181,-1.453893,-0.026652,0,4,3,13,-0.948410,-0.860717,1.134180,-1.324043,-0.536119,-0.561238,0,-0.470165,-0.023756,0
10218,0.043147,42,210,1045,0.647408,-0.071741,2,1,8,6,0.338490,0.386830,-1.108159,1.788302,0.658580,-0.561238,0,-0.470165,-0.023756,1
